In [1]:
import pytest
import ipytest

ipytest.config(rewrite_asserts=True, magics=True)

__file__ = 'parametrization.ipynb'

/home/mdexet/Workdir/EUCLID/WorkshopPythonCpp/Python-pytest/venv/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/home/mdexet/Workdir/EUCLID/WorkshopPythonCpp/Python-pytest/venv/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


# Parametrization

_Pytest_ has a very interesting feature of parametrization of test.

It respect the famous Don't Repeat Yourself.

In [2]:
# Silly function
def compute(n,m):
    if n <= 10 : 
        return n*m
    elif m >= 5 :
        return n*m*m
    else:
        return n+m

In [3]:
%%run_pytest[clean] -qq

def test_1_2():
    assert compute(1,2) == 2
    
def test_12_2(): 
    assert compute(12,2) == 14
    
def test_2_100():
    assert compute(2,100) == 200
        

...                                                                                                                                                                                                         [100%]


## Test expectations with @pytest.mark.parametrize

_Pytest_ allows to create several tests with 

`@pytest.mark.parametrize`

In [4]:
%%run_pytest[clean] -v

import pytest

@pytest.mark.parametrize("first_arg,second_arg,result",[(1,2,2), (12,2,14), (2,100,200)])
def test_it(first_arg,second_arg,result):
    assert compute(int(first_arg),int(second_arg)) == int(result)


=============================================================================================== test session starts ===============================================================================================
platform linux -- Python 3.6.8, pytest-5.2.2, py-1.8.0, pluggy-0.13.0 -- /home/mdexet/Workdir/EUCLID/WorkshopPythonCpp/Python-pytest/venv/bin/python3.6
cachedir: .pytest_cache
metadata: {'Python': '3.6.8', 'Platform': 'Linux-4.15.0-66-generic-x86_64-with-debian-buster-sid', 'Packages': {'pytest': '5.2.2', 'py': '1.8.0', 'pluggy': '0.13.0'}, 'Plugins': {'sugar': '0.9.2', 'cov': '2.8.1', 'mock': '1.11.2', 'html': '2.0.0', 'metadata': '1.8.0'}}
rootdir: /home/mdexet/Workdir/EUCLID/WorkshopPythonCpp/Python-pytest
plugins: sugar-0.9.2, cov-2.8.1, mock-1.11.2, html-2.0.0, metadata-1.8.0
collecting ... collected 3 items

parametrization.py::test_it[1-2-2] PASSED                                                                                                                             

Be aware of the **NODE id** names of performed tests:

```
parametrization.py::test_it[1-2-2] PASSED                                                                                                                                                                   [ 33%]
parametrization.py::test_it[12-2-14] PASSED                                                                                                                                                                 [ 66%]
parametrization.py::test_it[2-100-200] PASSED                                                                                                                                                               [100%]
```

***Be careful***

* First parameter of annotation is a string with names of joined comma-separated test arguments.
* Second parameter is a **list of tuples**
* Each **tuple** is populated with value of each arguments in the same order.

<center>
    <img src="images/pytest-param.png"/>
    </center>

## Cartesian product with @pytest.mark.parametrize

You could create a cartesian product if you specify this way:

In [5]:
%%run_pytest[clean] -v

@pytest.mark.parametrize("x",[1,2])
@pytest.mark.parametrize("y",[1,2])
def test_it(x,y):
    pass


=============================================================================================== test session starts ===============================================================================================
platform linux -- Python 3.6.8, pytest-5.2.2, py-1.8.0, pluggy-0.13.0 -- /home/mdexet/Workdir/EUCLID/WorkshopPythonCpp/Python-pytest/venv/bin/python3.6
cachedir: .pytest_cache
metadata: {'Python': '3.6.8', 'Platform': 'Linux-4.15.0-66-generic-x86_64-with-debian-buster-sid', 'Packages': {'pytest': '5.2.2', 'py': '1.8.0', 'pluggy': '0.13.0'}, 'Plugins': {'sugar': '0.9.2', 'cov': '2.8.1', 'mock': '1.11.2', 'html': '2.0.0', 'metadata': '1.8.0'}}
rootdir: /home/mdexet/Workdir/EUCLID/WorkshopPythonCpp/Python-pytest
plugins: sugar-0.9.2, cov-2.8.1, mock-1.11.2, html-2.0.0, metadata-1.8.0
collecting ... collected 4 items

parametrization.py::test_it[1-1] PASSED                                                                                                                               

In [6]:
import math

def distance(x,y,z):
    return math.sqrt(x*x + y*y + z*z)

def power_law(r,x,y,z):
    d = distance(x,y,z)
    if d < 1.6*r:
        return math.log(d)/(d*d)
    else:
        return 0.015

In [7]:
%%run_pytest[clean] -qq

@pytest.mark.parametrize("x",[1,5,10])
@pytest.mark.parametrize("y",[1,5,10])
@pytest.mark.parametrize("z",[1,5,10])
def test_power_law(x,y,z):
    assert power_law(5,x,y,z) > 0.01


...........................                                                                                                                                                                                 [100%]


## Fixture parametrization

Parametrization coud be done at the fixture level and so been shared among several test suites.

Imagine you have to perform a test suite about Euclid Organisation Units

In [8]:
organisational_unit_list=['EXT','LE3','MER','NIR','PHZ','SHE','SIM','VIS']

In [9]:
@pytest.fixture(params=organisational_unit_list)
def organisational_unit(request):
    return request.param

**NOTE** the parameters is passed through `request`, a [built-in fixture named FixtureClass](https://docs.pytest.org/en/latest/reference.html#_pytest.fixtures.FixtureRequest). 

Each parm is obtained from `request.param`.

In [10]:
%%run_pytest[clean] -v

def test_something_by_ou(organisational_unit):
    pass

def test_something_else_by_ou(organisational_unit):
    pass

=============================================================================================== test session starts ===============================================================================================
platform linux -- Python 3.6.8, pytest-5.2.2, py-1.8.0, pluggy-0.13.0 -- /home/mdexet/Workdir/EUCLID/WorkshopPythonCpp/Python-pytest/venv/bin/python3.6
cachedir: .pytest_cache
metadata: {'Python': '3.6.8', 'Platform': 'Linux-4.15.0-66-generic-x86_64-with-debian-buster-sid', 'Packages': {'pytest': '5.2.2', 'py': '1.8.0', 'pluggy': '0.13.0'}, 'Plugins': {'sugar': '0.9.2', 'cov': '2.8.1', 'mock': '1.11.2', 'html': '2.0.0', 'metadata': '1.8.0'}}
rootdir: /home/mdexet/Workdir/EUCLID/WorkshopPythonCpp/Python-pytest
plugins: sugar-0.9.2, cov-2.8.1, mock-1.11.2, html-2.0.0, metadata-1.8.0
collecting ... collected 16 items

parametrization.py::test_something_by_ou[EXT] PASSED                                                                                                                 

And now test something by Euclid countries

In [11]:
%%run_pytest[clean] -v

countries=['AT','DK','FR', 'FI', 'DE','IT','NL','NO', 'CH','PT','RO','UK','USA']

@pytest.fixture(params=countries)
def euclid_country(request):
    return request.param


def test_something_by_euclid_country(euclid_country):
    pass

=============================================================================================== test session starts ===============================================================================================
platform linux -- Python 3.6.8, pytest-5.2.2, py-1.8.0, pluggy-0.13.0 -- /home/mdexet/Workdir/EUCLID/WorkshopPythonCpp/Python-pytest/venv/bin/python3.6
cachedir: .pytest_cache
metadata: {'Python': '3.6.8', 'Platform': 'Linux-4.15.0-66-generic-x86_64-with-debian-buster-sid', 'Packages': {'pytest': '5.2.2', 'py': '1.8.0', 'pluggy': '0.13.0'}, 'Plugins': {'sugar': '0.9.2', 'cov': '2.8.1', 'mock': '1.11.2', 'html': '2.0.0', 'metadata': '1.8.0'}}
rootdir: /home/mdexet/Workdir/EUCLID/WorkshopPythonCpp/Python-pytest
plugins: sugar-0.9.2, cov-2.8.1, mock-1.11.2, html-2.0.0, metadata-1.8.0
collecting ... collected 13 items

parametrization.py::test_something_by_euclid_country[AT] PASSED                                                                                                      

And by Organisation Unit **and** by country

In [12]:
%%run_pytest[clean] -v

def test_something_by_euclid_country(organisational_unit, euclid_country):
    pass

=============================================================================================== test session starts ===============================================================================================
platform linux -- Python 3.6.8, pytest-5.2.2, py-1.8.0, pluggy-0.13.0 -- /home/mdexet/Workdir/EUCLID/WorkshopPythonCpp/Python-pytest/venv/bin/python3.6
cachedir: .pytest_cache
metadata: {'Python': '3.6.8', 'Platform': 'Linux-4.15.0-66-generic-x86_64-with-debian-buster-sid', 'Packages': {'pytest': '5.2.2', 'py': '1.8.0', 'pluggy': '0.13.0'}, 'Plugins': {'sugar': '0.9.2', 'cov': '2.8.1', 'mock': '1.11.2', 'html': '2.0.0', 'metadata': '1.8.0'}}
rootdir: /home/mdexet/Workdir/EUCLID/WorkshopPythonCpp/Python-pytest
plugins: sugar-0.9.2, cov-2.8.1, mock-1.11.2, html-2.0.0, metadata-1.8.0
collecting ... collected 104 items

parametrization.py::test_something_by_euclid_country[EXT-AT] PASSED                                                                                                 

parametrization.py::test_something_by_euclid_country[MER-CH] PASSED                                                                                                                                         [ 33%]
parametrization.py::test_something_by_euclid_country[MER-PT] PASSED                                                                                                                                         [ 34%]
parametrization.py::test_something_by_euclid_country[MER-RO] PASSED                                                                                                                                         [ 35%]
parametrization.py::test_something_by_euclid_country[MER-UK] PASSED                                                                                                                                         [ 36%]
parametrization.py::test_something_by_euclid_country[MER-USA] PASSED                                                                                        

parametrization.py::test_something_by_euclid_country[SHE-NO] PASSED                                                                                                                                         [ 70%]
parametrization.py::test_something_by_euclid_country[SHE-CH] PASSED                                                                                                                                         [ 71%]
parametrization.py::test_something_by_euclid_country[SHE-PT] PASSED                                                                                                                                         [ 72%]
parametrization.py::test_something_by_euclid_country[SHE-RO] PASSED                                                                                                                                         [ 73%]
parametrization.py::test_something_by_euclid_country[SHE-UK] PASSED                                                                                         

# Conclusion

Parametrization is an essential feature of Pytest

With little setup you can explore very large spaces of configuration.
